In [10]:
import torch
import sounddevice as sd
import numpy as np
from faster_whisper import WhisperModel
import time
from collections import deque
import threading
import torch
from vosk import Model, KaldiRecognizer
import json

TRANSCRIPTION_MODE = "vosk"  
# options: "vosk", "whisper"

vad_model, vad_utils = torch.hub.load(
    repo_or_dir = "snakers4/silero-vad",
    model = "silero_vad",
    force_reload = False
)

(get_speech_timestamps, save_audio, read_audio, VADIterator, collect_chunks) = vad_utils

Using cache found in C:\Users\PC/.cache\torch\hub\snakers4_silero-vad_master


In [11]:
SAMPLE_RATE = 16000
audio_buffer = []

def audio_callback(indata, frames, time, status):
    audio_buffer.append(indata.copy())

stream = sd.InputStream(
    samplerate=SAMPLE_RATE, 
    channels=1,
    callback=audio_callback
)


In [12]:
vad_iterator = VADIterator(vad_model)

silence_ms = 0
is_user_speaking = False

def process_vad(chunk):
    global silence_ms, is_user_speaking

    speech_prob = vad_iterator(chunk, SAMPLE_RATE)

    if speech_prob > 0.5:
        silence_ms = 0
        is_user_speaking = True
    else:
        silence_ms += int(len(chunk) / SAMPLE_RATE * 1000)
        if silence_ms > 800:
            is_user_speaking = False

    return is_user_speaking, silence_ms

In [13]:

whisper = None

if TRANSCRIPTION_MODE == "whisper":
    whisper = WhisperModel(
        "small",
        device="cpu",
        compute_type="int8"
    )

WHISPER_WINDOW_SEC = 1.2

In [14]:
vosk_model = None
vosk_rec = None
VOSK_MIN_SAMPLES = 3200  # 0.2 sec @ 16kHz


if TRANSCRIPTION_MODE == "vosk":
    vosk_model = Model("models/vosk-model-small-en-us-0.15")
    vosk_rec = KaldiRecognizer(vosk_model, 16000)
    vosk_rec.SetWords(True)

print(f"ASR mode: {TRANSCRIPTION_MODE}")


ASR mode: vosk


In [15]:
asr_audio = deque()
asr_lock = threading.Lock()
current_partial_text = ""
vosk_reset_requested = False

def float32_to_int16(audio):
    audio = np.clip(audio, -1.0, 1.0)
    return (audio * 32767).astype(np.int16)

def asr_worker():
    global current_partial_text
    global vosk_reset_requested

    if TRANSCRIPTION_MODE == "whisper":
        # ---------- WHISPER MODE ----------
        while True:
            time.sleep(0.7)

            with asr_lock:
                if not asr_audio:
                    continue

                now = time.time()
                recent = [
                    frame for frame, t in asr_audio
                    if now - t <= WHISPER_WINDOW_SEC
                ]

            if not recent:
                continue

            audio_np = np.concatenate(recent)

            segments, _ = whisper.transcribe(
                audio_np,
                language="en",
                vad_filter=False,
                beam_size=1,
                temperature=0.0
            )

            text = " ".join(seg.text for seg in segments).strip()
            if text and text != current_partial_text:
                current_partial_text = text
                print("📝 Partial:", text)

    else:
        if vosk_reset_requested:
            vosk_rec.Reset()
            vosk_reset_requested = False
            current_partial_text = ""
        # ---------- VOSK MODE ----------
        while True:
            time.sleep(0.05)

            with asr_lock:
                if not asr_audio:
                    continue

                frame, _ = asr_audio.popleft()
                if len(frame) < VOSK_MIN_SAMPLES:
                    continue
                pcm16 = float32_to_int16(frame)

            try:
                is_final = vosk_rec.AcceptWaveform(pcm16.tobytes())
            except Exception:
                continue  # drop bad frame safely
            
            if vosk_rec.AcceptWaveform(pcm16.tobytes()):
                res = json.loads(vosk_rec.Result())
                text = res.get("text", "").strip()
                if text:
                    print("📝 Final:", text)
                    current_partial_text = ""
            else:
                res = json.loads(vosk_rec.PartialResult())
                partial = res.get("partial", "").strip()
                if partial and partial != current_partial_text:
                    current_partial_text = partial
                    print("📝 Partial:", partial)




In [16]:
threading.Thread(
    target=asr_worker,
    daemon=True
).start()

print("Asr worker started")


Asr worker started


In [21]:
# -------- ANALYSIS BUFFERS --------
energy_history = deque(maxlen=15)       # RMS history (~300ms)
pause_history = deque(maxlen=5)          # pause lengths within turn
micro_spike_times = deque(maxlen=5)      # small resumption attempts

last_ai_interrupted = False
TRAILING_CONJUNCTIONS = {
    "and","or","but","because","so","that","which","who",
    "when","if","though","while"
}

OPEN_ENDED_PREFIXES = (
    "i think","i guess","i'm not sure","the thing is","it depends"
)

QUESTION_LEADINS = (
    "do you think","would you say","is it possible","can you"
)

SELF_REPAIR_MARKERS = (
    "i mean","actually","sorry","no wait"
)

FILLER_ENDINGS = (
    "uh","um","like","you know","kind of"
)

def lexical_bias(text: str) -> float:
    if not text:
        return 0.0

    t = text.lower().strip()
    words = t.split()

    score = 0.0

    # 6️⃣ trailing conjunction blocker
    if words[-1] in TRAILING_CONJUNCTIONS:
        score -= 1.0

    # 7️⃣ open-ended phrases
    if any(t.startswith(p) for p in OPEN_ENDED_PREFIXES):
        score -= 0.6

    # 8️⃣ question lead-ins
    if any(t.startswith(q) for q in QUESTION_LEADINS):
        score -= 0.5

    # 9️⃣ self-repair markers
    if any(m in t[-20:] for m in SELF_REPAIR_MARKERS):
        score -= 0.4

    # 🔟 filler ending guard
    if words[-1] in FILLER_ENDINGS:
        score -= 0.7

    return score

def energy_decay_score():
    if len(energy_history) < 5:
        return 0.0
    x = np.arange(len(energy_history))
    y = np.array(energy_history)
    slope = np.polyfit(x, y, 1)[0]
    return 0.8 if slope < -0.00015 else 0.0

def micro_resumption_penalty(now, elapsed):
    if elapsed > 900:      # late pause → ignore micro-resumptions
        return 0.0
    recent = [t for t in micro_spike_times if now - t < 0.8]
    return -0.6 if len(recent) >= 2 else 0.0



In [24]:
# --------
# REAL-TIME TURN-TAKING ENGINE (FIXED)
# --------

# -------- CONFIG --------
VAD_MIN_SAMPLES = 512
PAUSE_MS = 600
END_MS = 1200
SAFETY_TIMEOUT_MS = 2500  # Critical fallback
ENERGY_FLOOR = 0.015      # Increased to ignore mic noise
WHISPER_WINDOW_SEC = 3.0
CONFIDENCE_THRESHOLD = 1.2  # Reduced from 1.8

# -------- STATE --------
state = "IDLE"              # IDLE | SPEAKING | PAUSING
last_voice_time = None
last_ai_interrupted = False
current_partial_text = ""
vosk_reset_requested = False

# Buffers
vad_buffer = np.zeros(0, dtype=np.float32)
energy_history = deque(maxlen=15)
pause_history = deque(maxlen=5)
micro_spike_times = deque(maxlen=5)

stream.start()
print("🎙️ Real-time conversation test started")

try:
    while True:
        if not audio_buffer:
            time.sleep(0.01)
            continue

        # ---- collect audio ----
        chunk = audio_buffer.pop(0).astype(np.float32).flatten()
        vad_buffer = np.concatenate([vad_buffer, chunk])

        if len(vad_buffer) < VAD_MIN_SAMPLES:
            continue

        frame = vad_buffer[:VAD_MIN_SAMPLES]
        vad_buffer = vad_buffer[VAD_MIN_SAMPLES:]

        if len(frame) < VAD_MIN_SAMPLES:
            continue

        now = time.time()
        rms = np.sqrt(np.mean(frame ** 2))
        energy_history.append(rms)

        # ---- CORRECT REAL-TIME VAD ----
        # Use raw model output (probability), NOT VADIterator
        with torch.no_grad():
            vad_confidence = vad_model(
                torch.from_numpy(frame).unsqueeze(0),
                16000
            ).item()
        speech_started = vad_confidence > 0.5

        # Sustained voice: at least 3 of last 15 frames above floor
        sustained = sum(e > ENERGY_FLOOR for e in energy_history) >= 3

        # ---- MICRO-SPIKE DETECTION (for PAUSING state) ----
        if state == "PAUSING" and rms > ENERGY_FLOOR:
            micro_spike_times.append(now)

        # ---- STATE MACHINE ----
        if state == "IDLE":
            if speech_started or sustained:
                state = "SPEAKING"
                last_voice_time = now
                print("🟢 Speech started")

        elif state == "SPEAKING":
            if speech_started or sustained:
                last_voice_time = now
            else:
                elapsed = (now - last_voice_time) * 1000
                if elapsed >= PAUSE_MS:
                    state = "PAUSING"
                    print(f"🟡 Pause {int(elapsed)} ms")

        elif state == "PAUSING":
            elapsed = (now - last_voice_time) * 1000

            # 🔥 SAFETY TIMEOUT: Always end turn after 2.5s silence
            if elapsed > SAFETY_TIMEOUT_MS:
                print(f"🔴 SAFETY TIMEOUT: Force-ending turn after {elapsed:.0f}ms")
                state = "IDLE"
                last_voice_time = None
                energy_history.clear()
                pause_history.clear()
                micro_spike_times.clear()
                last_ai_interrupted = False
                with asr_lock:
                    asr_audio.clear()
                current_partial_text = ""
                if TRANSCRIPTION_MODE == "vosk":
                    vosk_reset_requested = True
                continue

            # Resume speech?
            if speech_started or sustained:
                state = "SPEAKING"
                last_voice_time = now
                print("🟢 Speech resumed")
            else:
                # Calculate turn-end confidence
                confidence = 0.0

                # 1. Silence duration
                if elapsed > END_MS:
                    confidence += 1.0

                # 2. Energy decay (consistently low energy)
                if len(energy_history) >= 8:
                    recent_energies = list(energy_history)[-8:]  # last ~160ms
                    if max(recent_energies) < ENERGY_FLOOR * 1.8:
                        confidence += 0.7

                # 3. Micro-resumption penalty (only for short pauses)
                if elapsed < 1000:
                    recent_spikes = [t for t in micro_spike_times if now - t < 0.6]
                    if len(recent_spikes) >= 2:
                        confidence -= 0.5

                # 4. Lexical bias (only block early endings)
                if elapsed < 900 and current_partial_text:
                    confidence += lexical_bias(current_partial_text) * 0.6

                # 5. Interruption memory
                if last_ai_interrupted:
                    confidence -= 0.5

                # End turn?
                if confidence >= CONFIDENCE_THRESHOLD:
                    print(f"🔴 Turn ended (confidence={confidence:.2f}, silence={elapsed:.0f}ms)")
                    state = "IDLE"
                    last_voice_time = None
                    energy_history.clear()
                    pause_history.clear()
                    micro_spike_times.clear()
                    last_ai_interrupted = False
                    with asr_lock:
                        asr_audio.clear()
                    current_partial_text = ""
                    if TRANSCRIPTION_MODE == "vosk":
                        vosk_reset_requested = True

        # ---- COLLECT AUDIO FOR ASR ----
        if state in ("SPEAKING", "PAUSING"):
            with asr_lock:
                if TRANSCRIPTION_MODE == "vosk":
                    # For Vosk: accumulate full chunks
                    if not hasattr(asr_worker, "vosk_buf"):
                        asr_worker.vosk_buf = np.zeros(0, dtype=np.float32)
                    asr_worker.vosk_buf = np.concatenate([asr_worker.vosk_buf, frame])
                    
                    # Feed Vosk in 0.2s+ chunks
                    while len(asr_worker.vosk_buf) >= VOSK_MIN_SAMPLES:
                        chunk_to_send = asr_worker.vosk_buf[:VOSK_MIN_SAMPLES]
                        asr_worker.vosk_buf = asr_worker.vosk_buf[VOSK_MIN_SAMPLES:]
                        asr_audio.append((chunk_to_send.copy(), now))
                else:
                    # For Whisper: stream all frames
                    asr_audio.append((frame.copy(), now))
                    # Trim old frames beyond window
                    cutoff = now - WHISPER_WINDOW_SEC
                    while asr_audio and asr_audio[0][1] < cutoff:
                        asr_audio.popleft()

except KeyboardInterrupt:
    stream.stop()
    print("\n🛑 Test stopped")

🎙️ Real-time conversation test started
🟢 Speech started
📝 Partial: hey
🟡 Pause 606 ms
📝 Final: hey
🟢 Speech resumed
📝 Partial: andre andre
📝 Partial: andre andre don't
📝 Partial: andre andre don't understand
📝 Partial: andre andre don't understand if
📝 Partial: andre andre don't understand and empathize
📝 Partial: andre andre don't understand if if if if if
📝 Partial: andre andre don't understand if if if if
📝 Partial: andre andre don't understand if if if if it
📝 Partial: andre andre don't understand if if if if you are working
📝 Partial: andre andre don't understand if if if if you are working world
📝 Partial: andre andre don't understand if if if if you are working world will
🟡 Pause 620 ms
📝 Final: andre andre don't own under threat and if if if if it is working very well
🔴 Turn ended (confidence=1.70, silence=1221ms)
🟢 Speech started
📝 Partial: see
📝 Partial: see them
📝 Partial: see them that
🟡 Pause 604 ms
📝 Final: see than that
🟢 Speech resumed
📝 Partial: the the
📝 Partial: the 